# Group cross sections from OpenMC

- Date: 21 march 2023
- Based on: Zsolts Serpent groupxs files
- Changelog:
    - 21 march 2023: Initial version

### Questions
1. ...

In [19]:
# Fuel properties
FUEL_TEMP = 900 # K
# FUEL_ENRICHMENT = 4.0 # wo% # NOT NEEDED IN THIS MODEL
# FUEL_R = 5 # cm # NOT NEEDED IN THIS MODEL

# Moderator properties
MODERATOR_TEMP = 600 # K
MODERATOR_R = 1.15 # cm

# Cladding properties
CLADDING_TEMP = 900 # K
CLADDING_R = 1.25 # cm

# Lattice properties
LATTICE_ELEMENTS_INNER = 7
LATTICE_PITCH_INNER = 3 # cm

# Plenum properties
PLENUM_HEIGHT = 25 # cm

# Reflective boundary parameters
REFLECTIVE_BOUNDARY_HALF_WIDTH = 7.5 # cm -> For the infinite lattice

# Core properties
CORE_HEIGHT = 350 # cm

# Run properties
INACTIVE_BATCHES = 50
PARTICLE_COUNT = 30000
ACTIVE_BATCH_COUNT = 300
FAST_REACTOR = True # True if epithermal, False if thermal

DO_PLOT = True

In [20]:
import os
import sys
TOOLS_PATH = f"{os.getenv('MASTER_PROJECT_ROOT_FOLDER')}/logistics"
sys.path.insert(0, TOOLS_PATH)
import openmc
import numpy as np
import matplotlib.pyplot as plt
from tools import run_tools, nuclear_data_tools
import time
import random
import openmc.mgxs as mgxs
import pickle
import pandas as pd

df = pd.DataFrame()

for FUEL_ONLY in [False, True]:
    for FAST_REACTOR in [True, False]:
    # for FAST_REACTOR in [True]:
        # FAST_REACTOR = False # True if epithermal, False if thermal
        NEUTRON_TEMP = "EPITHERMAL" if FAST_REACTOR else "THERMAL"
        print(NEUTRON_TEMP)

        # @@@@@@@@@@@@@@@@@@@@@@ Materials @@@@@@@@@@@@@@@@@@@@@@
        # Fuel
        material_fuel = openmc.Material(1, "ULiF", temperature=FUEL_TEMP)
        material_fuel.add_nuclide('U235', 3.11e-2, 'wo')
        material_fuel.add_nuclide('U238', 5.909e-1, 'wo')
        material_fuel.add_nuclide('Li7', 4.8358e-2, 'wo')
        material_fuel.add_nuclide('Li6', 2.418e-6, 'wo')
        material_fuel.add_nuclide('F19', 3.2964e-1, 'wo')
        material_fuel.set_density('g/cm3', 5.01)

        # Cladding
        material_cladding = openmc.Material(3, "CSi", temperature=CLADDING_TEMP)
        material_cladding.add_nuclide('C0', 2.9936e-1, 'wo')
        material_cladding.add_nuclide('Si28', 6.4365e-1, 'wo')
        material_cladding.add_nuclide('Si29', 3.3866e-2, 'wo')
        material_cladding.add_nuclide('Si30', 2.3120e-2, 'wo')
        material_cladding.set_density('g/cm3', 3.21)

        # Moderator
        material_moderator = openmc.Material(2, "ZrH", temperature=MODERATOR_TEMP)
        material_moderator.add_nuclide('Zr90', 4.9793e-1, 'wo')
        material_moderator.add_nuclide('Zr91', 1.0980e-1, 'wo')
        material_moderator.add_nuclide('Zr92', 1.6967e-1, 'wo')
        material_moderator.add_nuclide('Zr94', 1.7569e-1, 'wo')
        material_moderator.add_nuclide('Zr96', 2.8908e-2, 'wo')
        material_moderator.add_nuclide('H1', 1.8007e-2, 'wo')
        material_moderator.add_nuclide('H2', 4.1389e-6, 'wo')
        material_moderator.set_density('g/cm3', 5.66)

        material_moderator.add_s_alpha_beta('c_H_in_ZrH') # DID NOT EXIST AT OR NEAR 900 K

        # Create Materials list and export to XML
        mats = openmc.Materials([material_fuel, material_cladding, material_moderator])
        mats.export_to_xml()
        # @@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@



        # @@@@@@@@@@@@@@@@@@@@@@ Geometry @@@@@@@@@@@@@@@@@@@@@@
        # Planes
        plane_bottom = openmc.ZPlane(z0=-CORE_HEIGHT/2, boundary_type='vacuum')
        plane_top = openmc.ZPlane(z0=CORE_HEIGHT/2, boundary_type='vacuum')

        # Cylinders
        r_moderator = openmc.ZCylinder(r=MODERATOR_R)
        r_cladding = openmc.ZCylinder(r=CLADDING_R)

        # Regions
        region_height_bound = +plane_bottom & -plane_top
        region_moderator = -r_moderator & region_height_bound
        region_cladding = +r_moderator & -r_cladding & region_height_bound
        region_fuel = +r_cladding & region_height_bound

        # # Cells
        cell_cladding = openmc.Cell(name="cladding", fill=material_cladding, region=region_cladding)
        cell_moderator = openmc.Cell(name="moderator", fill=material_moderator, region=region_moderator)
        cell_fuel = openmc.Cell(name="fuel", fill=material_fuel, region=region_fuel)

        # # Universe
        universe_pin = openmc.Universe(cells=(cell_cladding, cell_moderator, cell_fuel))

        # # Outer universe
        cell_all_fuel = openmc.Cell(name="outer", fill=material_fuel)
        universe_outer = openmc.Universe(cells=(cell_all_fuel,))
        # @@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@



        # @@@@@@@@@@@@@@@@@@@@@@ Lattice @@@@@@@@@@@@@@@@@@@@@@
        # Inner lattice
        innerLatticeDistanceToCenter = LATTICE_PITCH_INNER * LATTICE_ELEMENTS_INNER / 2
        lattice_inner = openmc.RectLattice()
        lattice_inner.lower_left = (-innerLatticeDistanceToCenter, -innerLatticeDistanceToCenter) # TODO: Is this needed?
        # lattice_inner.lower_left = (0, 0)
        lattice_inner.pitch = (LATTICE_PITCH_INNER, LATTICE_PITCH_INNER)

        if not FAST_REACTOR: # Thermal
            inner_lattice_string =   """F F F F F F F
                                        F M M M M M F
                                        F M M M M M F
                                        F M M M M M F
                                        F M M M M M F
                                        F M M M M M F
                                        F F F F F F F"""
        else: # Epithermal
            inner_lattice_string =   """F F F F F F F
                                        F F F F F F F
                                        F F M F M F F
                                        F F F F F F F
                                        F F M F M F F
                                        F F F F F F F
                                        F F F F F F F"""
            
        if FUEL_ONLY:
            inner_lattice_string =   """F F F F F F F
                                        F F F F F F F
                                        F F F F F F F
                                        F F F F F F F
                                        F F F F F F F
                                        F F F F F F F
                                        F F F F F F F"""
            
        lattice_inner.universes = [[universe_outer if cell == 'F' else universe_pin for cell in row.split()]
                                    for row in inner_lattice_string.splitlines()]

        # Make a universe out of the inner lattice
        cell_lattice_inner = openmc.Cell(name="lattice_inner", fill=lattice_inner)
        universe_lattice_inner = openmc.Universe(cells=(cell_lattice_inner,))

        plane_plenum_bottom = openmc.ZPlane(z0=-CORE_HEIGHT/2+PLENUM_HEIGHT)
        plane_plenum_top = openmc.ZPlane(z0=CORE_HEIGHT/2-PLENUM_HEIGHT)

        # Make planes on the west, east, north, and south sides of the lattice with reflective boundary conditions
        plane_west = openmc.XPlane(x0=-REFLECTIVE_BOUNDARY_HALF_WIDTH, boundary_type='reflective')
        plane_east = openmc.XPlane(x0=REFLECTIVE_BOUNDARY_HALF_WIDTH, boundary_type='reflective')
        plane_south = openmc.YPlane(y0=-REFLECTIVE_BOUNDARY_HALF_WIDTH, boundary_type='reflective')
        plane_north = openmc.YPlane(y0=REFLECTIVE_BOUNDARY_HALF_WIDTH, boundary_type='reflective')

        region_within_lattice = +plane_west & -plane_east & +plane_south & -plane_north

        bottom_slab = +plane_bottom & -plane_plenum_bottom
        top_slab = -plane_top & +plane_plenum_top
        region_height_bound_inner = +plane_plenum_bottom & -plane_plenum_top
        # region_containment = (+r_containment_inner & -r_containment_outer & region_height_bound)
        region_plenum = region_within_lattice & (top_slab | bottom_slab)
        region_main = region_within_lattice & region_height_bound_inner

        # cell_containment = openmc.Cell(name="containment", fill=material_cladding, region=region_containment)
        cell_main = openmc.Cell(fill=lattice_inner, region=region_main)
        cell_plenum = openmc.Cell(name="plenum", fill=material_fuel, region=region_plenum)
        geometry = openmc.Geometry([cell_main, cell_plenum])
        geometry.export_to_xml()
        # @@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@



        # @@@@@@@@@@@@@@@@@@@@@@ Plots @@@@@@@@@@@@@@@@@@@@@@
        if DO_PLOT:
            PLOT_COLOR_BY = 'material'
            PLOT_PIXELS = (1000, 1000)
            PLOT_FUEL_COLOR = 'yellowgreen'
            PLOT_CLADDING_COLOR = 'lightslategray'
            PLOT_MODERATOR_COLOR = 'midnightblue'

            colors = {
                material_fuel: PLOT_FUEL_COLOR,
                material_cladding: PLOT_CLADDING_COLOR,
                material_moderator: PLOT_MODERATOR_COLOR
            }

            slize_coord_XY = 0

            plotXY = openmc.Plot.from_geometry(geometry, basis='xy', slice_coord=slize_coord_XY)
            plotXY.color_by = PLOT_COLOR_BY
            plotXY.filename = f'plotXY-{NEUTRON_TEMP}'
            if FUEL_ONLY:
                plotXY.filename = f'plotXY-FUEL_ONLY'
            plotXY.pixels = PLOT_PIXELS
            plotXY.colors = colors
            plotXY.to_ipython_image()

            # Add slice_coord to actually see the moderator rods in the plot
            plotXZ = openmc.Plot.from_geometry(geometry, basis='xz', slice_coord=4)
            plotXZ.color_by = PLOT_COLOR_BY
            plotXZ.filename = f'plotXZ-{NEUTRON_TEMP}'
            if FUEL_ONLY:
                plotXZ.filename = f'plotXZ-FUEL_ONLY'
            plotXZ.pixels = PLOT_PIXELS
            plotXZ.colors = colors
            plotXZ.to_ipython_image()
        # @@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@


        # @@@@@@@@@@@@@@@@@@@@@@ Tallies @@@@@@@@@@@@@@@@@@@@@@
        # 2G cross sections
        G2 = openmc.mgxs.EnergyGroups()
        G2.group_edges = np.array([0.0, 0.625, 1e43])

        # Create the openmc.RegularMesh object to be used as the domain for the MGXS
        # G2_domain = openmc.RegularMesh()
        # G2_domain.dimension = [1, 1, 1]
        # G2_domain.lower_left = [-innerLatticeDistanceToCenter, -innerLatticeDistanceToCenter, -CORE_HEIGHT/2]
        # G2_domain.upper_right = [innerLatticeDistanceToCenter, innerLatticeDistanceToCenter, CORE_HEIGHT/2]
        G2_domain = geometry.root_universe

        # tot_1 = mgxs.TotalXS(domain=G2_domain, energy_groups=G2)
        transport_1  = mgxs.TransportXS(domain=G2_domain, energy_groups=G2)
        abs_1 = mgxs.AbsorptionXS(domain=G2_domain, energy_groups=G2)
        nufiss_1 = mgxs.FissionXS(domain=G2_domain, energy_groups=G2, nu=True)
        fiss_1 = mgxs.FissionXS(domain=G2_domain, energy_groups=G2)
        chi_1 = mgxs.Chi(domain=G2_domain, energy_groups=G2)
        scatter_1 = mgxs.ScatterMatrixXS(domain=G2_domain, energy_groups=G2)

        group_tallies = [transport_1, abs_1, nufiss_1, fiss_1, chi_1, scatter_1]

        tallies = openmc.Tallies([])
        for group_tally in group_tallies:
            tallies += group_tally.tallies.values()
        tallies.export_to_xml()
        # @@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@


        USE_SAMPLED_DATA = False
        # @@@@@@@@@@@@@@@@@@@@@@ Perturbations @@@@@@@@@@@@@@@@@@@@@@
        nuclear_data_tools.restore_F19_h5()
        # @@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@


        # @@@@@@@@@@@@@@@@@@@@@@ Settings @@@@@@@@@@@@@@@@@@@@@@
        point = openmc.stats.Point((0, 0, 0))
        src = openmc.Source(space=point)

        # # Shannon Entropy
        # regular_mesh = openmc.RegularMesh()
        # regular_mesh.lower_left, regular_mesh.upper_right = geometry.bounding_box
        # regular_mesh.dimension = [20, 20, 20]

        settings = openmc.Settings()
        settings.seed = random.randint(1, 1e10)
        settings.source = src
        # settings.entropy_mesh = regular_mesh
        settings.batches = ACTIVE_BATCH_COUNT + INACTIVE_BATCHES
        settings.inactive = INACTIVE_BATCHES
        settings.particles = PARTICLE_COUNT
        settings.export_to_xml()
        # @@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@



        # @@@@@@@@@@@@@@@@@@@@@@ Run OpenMC @@@@@@@@@@@@@@@@@@@@@@
        os.system('rm s*h5') # Remove old files
        t_start = time.time()
        openmc.run()
        t_end = time.time()
        # @@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@



        # @@@@@@@@@@@@@@@@@@@@@@ Save data @@@@@@@@@@@@@@@@@@@@@@
        run_settings = {
            "fuel_only": FUEL_ONLY,
            "fast_reactor": FAST_REACTOR,
            "neutron_temp": NEUTRON_TEMP,
            "use_sampled_data": USE_SAMPLED_DATA,
            "particle_count": PARTICLE_COUNT,
            "active_batch_count": ACTIVE_BATCH_COUNT,
            "inactive_batch_count": INACTIVE_BATCHES,
            "t_start": round(t_start, 3),
            "t_end": round(t_end, 3),
            "t_elapsed": round(t_end - t_start, 3),
            "seed": settings.seed
        }
        run_tools.dump_run_settings_json(run_settings)
        run_tools.save_run_data()
        # @@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@



        # @@@@@@@@@@@@@@@@@@@@@@ Group XS @@@@@@@@@@@@@@@@@@@@@@
        statepoint_number = run_settings["active_batch_count"] + run_settings["inactive_batch_count"]
        sp = openmc.StatePoint(f"./statepoint.{statepoint_number}.h5")

        

        # print(sp.tallies)
        for group_tally in group_tallies:
            group_tally.load_from_statepoint(sp)
            group_tally.print_xs()

            df_group = group_tally.get_pandas_dataframe()

            df_group["rxn_type"] = group_tally.rxn_type
            if group_tally.rxn_type == "fission": df_group["nu"] = group_tally.nu
               
            df_group["fuel_only"] = FUEL_ONLY
            if not FUEL_ONLY: 
                df_group["fast_reactor"] = FAST_REACTOR

            # print(df.head())
            
            df = pd.concat([df, df_group], ignore_index=True)
            
        # @@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@ 

        # If only doing fuel, dont need to do another run (for thermal/epithermal), is its the same
        if FUEL_ONLY:
            break

pkl_filename = "mgxs/mgxs_results.pkl"
with open(pkl_filename, "wb") as f:
    pickle.dump(df, f)

EPITHERMAL


/home/fne23_stjarnholm/miniconda3/envs/openmc-env/lib/python3.11/site-packages/openmc/mixin.py:70: IDWarning: Another Material instance already exists with id=1.
  warn(msg, IDWarning)
/home/fne23_stjarnholm/miniconda3/envs/openmc-env/lib/python3.11/site-packages/openmc/mixin.py:70: IDWarning: Another Material instance already exists with id=3.
  warn(msg, IDWarning)
/home/fne23_stjarnholm/miniconda3/envs/openmc-env/lib/python3.11/site-packages/openmc/mixin.py:70: IDWarning: Another Material instance already exists with id=2.
  warn(msg, IDWarning)
/home/fne23_stjarnholm/miniconda3/envs/openmc-env/lib/python3.11/site-packages/openmc/mixin.py:70: IDWarning: Another Filter instance already exists with id=325.
  warn(msg, IDWarning)
/home/fne23_stjarnholm/miniconda3/envs/openmc-env/lib/python3.11/site-packages/openmc/mixin.py:70: IDWarning: Another Filter instance already exists with id=326.
  warn(msg, IDWarning)
/home/fne23_stjarnholm/miniconda3/envs/openmc-env/lib/python3.11/site-packa

Restored /home/fne23_stjarnholm/jeff-3.3-hdf5/F19.h5 from /home/fne23_stjarnholm/jeff-3.3-hdf5/F19.h5.tmp
                                %%%%%%%%%%%%%%%
                           %%%%%%%%%%%%%%%%%%%%%%%%
                        %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
                      %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
                    %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
                   %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
                                    %%%%%%%%%%%%%%%%%%%%%%%%
                                     %%%%%%%%%%%%%%%%%%%%%%%%
                 ###############      %%%%%%%%%%%%%%%%%%%%%%%%
                ##################     %%%%%%%%%%%%%%%%%%%%%%%
                ###################     %%%%%%%%%%%%%%%%%%%%%%%
                ####################     %%%%%%%%%%%%%%%%%%%%%%
                #####################     %%%%%%%%%%%%%%%%%%%%%
                ######################     %%%%%%%%%%%%%%%%%%%%
                #######################     %%%%%%%%%%%%%%

cp: omitting directory ‘./mgxs’
cp: omitting directory ‘./results’


Data saved to results/RUNDATA_OpenMC_groupxs_52!
Multi-Group XS
	Reaction Type  =	transport
	Domain Type    =	universe
	Domain ID      =	95
	Cross Sections [cm^-1]:
            Group 1 [0.625      - 1e+43     eV]:	3.42e-01 +/- 3.16e-02%
            Group 2 [0.0        - 0.625     eV]:	6.11e-01 +/- 1.76e-01%



Multi-Group XS
	Reaction Type  =	absorption
	Domain Type    =	universe
	Domain ID      =	95
	Cross Sections [cm^-1]:
            Group 1 [0.625      - 1e+43     eV]:	8.53e-03 +/- 3.92e-02%
            Group 2 [0.0        - 0.625     eV]:	8.35e-02 +/- 1.39e-01%



Multi-Group XS
	Reaction Type  =	nu-fission
	Domain Type    =	universe
	Domain ID      =	95
	Cross Sections [cm^-1]:
            Group 1 [0.625      - 1e+43     eV]:	6.66e-03 +/- 4.00e-02%
            Group 2 [0.0        - 0.625     eV]:	1.51e-01 +/- 1.40e-01%



Multi-Group XS
	Reaction Type  =	fission
	Domain Type    =	universe
	Domain ID      =	95
	Cross Sections [cm^-1]:
            Group 1 [0.625      - 1e+43     eV

...

/home/fne23_stjarnholm/miniconda3/envs/openmc-env/lib/python3.11/site-packages/openmc/tallies.py:1255: RuntimeWarning: invalid value encountered in divide
  data = self.std_dev[indices] / self.mean[indices]
/home/fne23_stjarnholm/miniconda3/envs/openmc-env/lib/python3.11/site-packages/openmc/mixin.py:70: IDWarning: Another Material instance already exists with id=1.
  warn(msg, IDWarning)
/home/fne23_stjarnholm/miniconda3/envs/openmc-env/lib/python3.11/site-packages/openmc/mixin.py:70: IDWarning: Another Material instance already exists with id=3.
  warn(msg, IDWarning)
/home/fne23_stjarnholm/miniconda3/envs/openmc-env/lib/python3.11/site-packages/openmc/mixin.py:70: IDWarning: Another Material instance already exists with id=2.
  warn(msg, IDWarning)
/home/fne23_stjarnholm/miniconda3/envs/openmc-env/lib/python3.11/site-packages/openmc/mixin.py:70: IDWarning: Another Filter instance already exists with id=363.
  warn(msg, IDWarning)
/home/fne23_stjarnholm/miniconda3/envs/openmc-env/lib

Restored /home/fne23_stjarnholm/jeff-3.3-hdf5/F19.h5 from /home/fne23_stjarnholm/jeff-3.3-hdf5/F19.h5.tmp
                                %%%%%%%%%%%%%%%
                           %%%%%%%%%%%%%%%%%%%%%%%%
                        %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
                      %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
                    %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
                   %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
                                    %%%%%%%%%%%%%%%%%%%%%%%%
                                     %%%%%%%%%%%%%%%%%%%%%%%%
                 ###############      %%%%%%%%%%%%%%%%%%%%%%%%
                ##################     %%%%%%%%%%%%%%%%%%%%%%%
                ###################     %%%%%%%%%%%%%%%%%%%%%%%
                ####################     %%%%%%%%%%%%%%%%%%%%%%
                #####################     %%%%%%%%%%%%%%%%%%%%%
                ######################     %%%%%%%%%%%%%%%%%%%%
                #######################     %%%%%%%%%%%%%%

cp: omitting directory ‘./mgxs’
cp: omitting directory ‘./results’


Data saved to results/RUNDATA_OpenMC_groupxs_54!
Multi-Group XS
	Reaction Type  =	transport
	Domain Type    =	universe
	Domain ID      =	105
	Cross Sections [cm^-1]:
            Group 1 [0.625      - 1e+43     eV]:	3.27e-01 +/- 2.76e-02%
            Group 2 [0.0        - 0.625     eV]:	3.51e-01 +/- 1.08e+01%



Multi-Group XS
	Reaction Type  =	absorption
	Domain Type    =	universe
	Domain ID      =	105
	Cross Sections [cm^-1]:
            Group 1 [0.625      - 1e+43     eV]:	6.61e-03 +/- 3.79e-02%
            Group 2 [0.0        - 0.625     eV]:	5.80e-02 +/- 1.12e+01%



Multi-Group XS
	Reaction Type  =	nu-fission
	Domain Type    =	universe
	Domain ID      =	105
	Cross Sections [cm^-1]:
            Group 1 [0.625      - 1e+43     eV]:	4.42e-03 +/- 3.96e-02%
            Group 2 [0.0        - 0.625     eV]:	1.08e-01 +/- 1.12e+01%



Multi-Group XS
	Reaction Type  =	fission
	Domain Type    =	universe
	Domain ID      =	105
	Cross Sections [cm^-1]:
            Group 1 [0.625      - 1e+43   

/home/fne23_stjarnholm/miniconda3/envs/openmc-env/lib/python3.11/site-packages/openmc/tallies.py:1255: RuntimeWarning: invalid value encountered in divide
  data = self.std_dev[indices] / self.mean[indices]


In [21]:
# print("Importing...")
# from multiprocessing import Pool
# import importlib
# from tqdm import tqdm
# importlib.reload(run_tools)
# PROCESSES_COUNT = 1
# results = []

# result_folders = run_tools.get_all_result_dirs(dir_prefix="./")
# print(f"Found {len(result_folders)} result folders")

# def handle_folder(result_folder):
#     result_path = f"./results/{result_folder}"
#     run_settings = run_tools.load_run_settings_json(result_path)

#     statepoint_number = run_settings["active_batch_count"] + run_settings["inactive_batch_count"]
#     sp = openmc.StatePoint(f"{result_path}/statepoint.{statepoint_number}.h5")

#     result = {
#         "k-eff": sp.keff.nominal_value, 
#         "k-eff-std": sp.keff.std_dev,  
#         "result_path": result_path,
#     }

#     result.update(run_settings)

#     # FAST_REACTOR = False # True if epithermal, False if thermal
#     NEUTRON_TEMP = "EPITHERMAL" if result["fast_reactor"] else "THERMAL"
#     print(NEUTRON_TEMP, end="\n\n")

    
#     return result

# print(f"Loading data with {PROCESSES_COUNT} processes...")
# with Pool(PROCESSES_COUNT) as p:
#     # Loop over each run number and get the run settings as well as the k-effective
#     # for result_folder in tqdm(result_folders):
#     results = list(p.imap(handle_folder, result_folders))
